##### Baseline:      
    1) Dimensionality reduction to capture principal modes of app-usage
    2) Cluster activities based on clusters in the principal dimensions

### Get cleaned data

In [10]:
from importlib import reload
from utils import *
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import pickle
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
plotly.tools.set_credentials_file(username='AbdelWahabTurkmani', api_key='zTY632QGUJbqSVFms8bQ')

In [9]:
# time_percentage = 0.9
# explained_variance = 0.9
# df = pd.read_csv("data/rescuetime_data-ac-min.csv")
# data_pd = Clean_DF(df)
# data_pd.clean_data(time_percentage=time_percentage)
# data_pd.clean_df = data_pd.clean_df.reset_index()
# data_pd.get_pca(explained_variance=explained_variance)
# data_pd.get_day_time()
# obj0, obj1, obj2 are created here...

# Saving the objects:
# with open('tsne_results_pca.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump(tsne_results_pca, f)

# Getting back the objects:
with open('data_pd.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    data_pd = pickle.load(f)


In [5]:
print("Dataset size:", data_pd.clean_df.shape,'\n')
print("Number of apps that consume", time_percentage*100, "% of all users time: ",len(data_pd.popular_apps), '\n')
print("Cleaned dataset columns:",'\n', data_pd.clean_df.columns.values, '\n')
print("Number of components that explain", explained_variance*100,"% of the data: ",data_pd.pca_data.shape[1], '\n')

Dataset size: (16704, 9) 

Number of apps that consume 90.0 % of all users time:  99 

Cleaned dataset columns: 
 ['Date' 'Time Spent (seconds)' 'Activity' 'Category' 'Productivity'
 'Activity Vector' 'Productivity Score' 'Day' 'Time'] 

Number of components that explain 90.0 % of the data:  30 



### Compute Principal Components and visualize top-3 modes

In [6]:
data_pd.get_pca(explained_variance=explained_variance)

c = data_pd.clean_df['Productivity Score']
x = data_pd.pca_data[:,0]
y = data_pd.pca_data[:,1]
z = data_pd.pca_data[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-3 Visualization')

### Visualize entire PCA dimensional space using t-SNE

In [7]:
# tsne_pca = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
# tsne_results_pca = tsne_pca.fit_transform(data_pd.pca_data)

with open('data_pd.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    data_pd = pickle.load(f)

In [8]:
c = data_pd.clean_df['Productivity Score']
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='RdYlGn',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE PCA 90% variance Visualization')

High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~AbdelWahabTurkmani/0 or inside your plot.ly account where it is named 't-SNE PCA 90% variance Visualization'


## Clustering data in PCA dimensions

### Clustering data using the elbow method

    1) Run Kmeans for N iterations recording the inertia for each
    2) At each iteration save the error
    3) When viewing the errors, pick the iteration that is at the elbow of the error curve


In [11]:
inertia = np.zeros(25)
for i in range(2,26):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data_pd.pca_data)
    inertia[i-2] = kmeans.inertia_

In [12]:
trace = go.Scatter(x=[i for i in range(0,25)], y=inertia, mode='markers')
data= [trace]
py.iplot(data, filename='K-Means inerta')

### Visualize data projected onto top-3 PCA components with k-Means labels

In [15]:
kmeans = KMeans(n_clusters=7)
kmeans.fit(data_pd.pca_data)
print(set(kmeans.labels_))
for i in range(0,7):
    print(list(kmeans.labels_).count(i))

{0, 1, 2, 3, 4, 5, 6}
161
910
117
649
14190
613
64


In [16]:
c = kmeans.labels_
x = data_pd.pca_data[:,0]
y = data_pd.pca_data[:,1]
z = data_pd.pca_data[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]
# for i in range(0,len(t)):
#     t[i] = str(kmeans.labels_[i]) + '---' + t[i]
    
trace1 = go.Scatter3d(x=x,y=y,z=z,text=t,mode='markers',marker=dict(size=12,color=c, colorscale='Viridis',opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='PCA-3 Visualization with k-Means')

### Visualize PCA data using t-SNE  with k-Means labels

In [ ]:
c = kmeans.labels_
x = tsne_results_pca[:,0]
y = tsne_results_pca[:,1]
z = tsne_results_pca[:,2]
# t = data_pd.clean_df['Activity']
# t = data_pd.clean_df['Activity'].tolist()
# t = ['-'.join(x) for x in t]
# for i in range(0,len(t)):
#     t[i] = str(kmeans.labels_[i]) + '---' + t[i]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t, mode='markers',marker=dict(size=12,color=c, colorscale = 'Viridis', opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE PCA with k-Means')

### Visualize original data with k-Means clusters computed in PCA dimensions

In [ ]:
tsne30_orig = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
tsne_results30_orig = tsne30_orig.fit_transform(data_pd.activity_vector)

In [ ]:
c = kmeans.labels_
x = tsne_results30_orig[:,0]
y = tsne_results30_orig[:,1]
z = tsne_results30_orig[:,2]
t = data_pd.clean_df['Activity']
t = data_pd.clean_df['Activity'].tolist()
t = ['-'.join(x) for x in t]
for i in range(0,len(t)):
    t[i] = str(kmeans.labels_[i]) + '---' + t[i]

trace1 = go.Scatter3d(x=x,y=y,z=z,text=t, mode='markers',marker=dict(size=12,color=c, colorscale = 'Viridis', opacity=0.8))
data = [trace1]
layout = go.Layout(margin=dict(l=0,r=0,b=0,t=0))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE with full-D k-Means')

In [ ]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=15, kernel='rbf', degree=5)
kpca.fit(data_pd.activity_vector)
pca_kernel = kpca.transform(data_pd.activity_vector)

In [ ]:
inertia = np.zeros(25)
for i in range(2,26):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data_pd.pca_data)
    inertia[i-2] = kmeans.inertia_